<a href="https://colab.research.google.com/github/jeongjinjoo/Big-Data-practice/blob/main/%EC%9D%B4%EC%A7%81%EC%97%AC%EB%B6%80%20%ED%8C%90%EB%8B%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

데이터 설명 : 이직여부 판단 데이터 (target: 1: 이직 , 0 : 이직 x)

In [3]:
import pandas as pd
#데이터 로드
x_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/HRdata/X_train.csv")
y_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/HRdata/y_train.csv")
x_test= pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/HRdata/X_test.csv")


display(x_train.head())
display(y_train.head())

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours
0,25298,city_138,0.836,Male,No relevent experience,Full time course,High School,NaN,5,100-500,Pvt Ltd,1,45
1,4241,city_160,0.920,Male,No relevent experience,Full time course,High School,NaN,5,NaN,NaN,1,17
2,24086,city_57,0.866,Male,No relevent experience,no_enrollment,Graduate,STEM,10,NaN,NaN,1,50
3,26773,city_16,0.910,Male,Has relevent experience,no_enrollment,Graduate,STEM,>20,50-99,Pvt Ltd,>4,135
4,32325,city_143,0.740,NaN,No relevent experience,Full time course,Graduate,STEM,5,NaN,NaN,never,17


,enrollee_id,target
0,25298,0.0
1,4241,1.0
2,24086,0.0
3,26773,0.0
4,32325,1.0


In [4]:
print(x_train.info()) # 결측치? gender, enrolled_university, education_level, major_Discipline, company_size 등
# 결측치가 있어도 따로 변환 x, dummies화 시
print(y_train.info()) #taget이 종속변수

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12452 entries, 0 to 12451
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   enrollee_id             12452 non-null  int64  
 1   city                    12452 non-null  object 
 2   city_development_index  12452 non-null  float64
 3   gender                  9535 non-null   object 
 4   relevent_experience     12452 non-null  object 
 5   enrolled_university     12195 non-null  object 
 6   education_level         12137 non-null  object 
 7   major_discipline        10586 non-null  object 
 8   experience              12415 non-null  object 
 9   company_size            8600 non-null   object 
 10  company_type            8471 non-null   object 
 11  last_new_job            12179 non-null  object 
 12  training_hours          12452 non-null  int64  
dtypes: float64(1), int64(2), object(10)
memory usage: 1.2+ MB
None
<class 'pandas.core.frame.Da

In [5]:
print(x_train.nunique())
print(x_train.shape) # 12452행 13열
print(y_train.shape) # 2행 12452행

enrollee_id               12452
city                        122
city_development_index       92
gender                        3
relevent_experience           2
enrolled_university           3
education_level               5
major_discipline              6
experience                   22
company_size                  8
company_type                  6
last_new_job                  6
training_hours              241
dtype: int64
(12452, 13)
(12452, 2)


In [6]:
print(x_train.isnull().sum()) #gender : 2917, 등등
print(x_train.info())

enrollee_id                  0
city                         0
city_development_index       0
gender                    2917
relevent_experience          0
enrolled_university        257
education_level            315
major_discipline          1866
experience                  37
company_size              3852
company_type              3981
last_new_job               273
training_hours               0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12452 entries, 0 to 12451
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   enrollee_id             12452 non-null  int64  
 1   city                    12452 non-null  object 
 2   city_development_index  12452 non-null  float64
 3   gender                  9535 non-null   object 
 4   relevent_experience     12452 non-null  object 
 5   enrolled_university     12195 non-null  object 
 6   education_level         12137 non-null  object

In [10]:
#범주형 변수면서 적당히 많은 unique값을 가진 컬럼은 날림.
drop_col = ['enrollee_id','city','company_type','experience']
x_train_drop = x_train.drop(columns = drop_col)
x_test_drop = x_test.drop(columns = drop_col)

from sklearn.model_selection import train_test_split
x_train_dummies = pd.get_dummies(x_train_drop)
y = y_train['target'].astype('int')

x_test_dummies = pd.get_dummies(x_test_drop)
x_test_dummies = x_test_dummies[x_train_dummies.columns]

X_train, X_val, Y_train, Y_val = train_test_split(x_train_dummies, y ,test_size = 0.2, random_state = 42)

from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(random_state = 24)
rfc.fit(X_train, Y_train)

from sklearn.metrics import accuracy_score, roc_auc_score

predict_train_label =rfc.predict(X_train)
predict_train_proba = rfc.predict_proba(X_train)[:,1]

predict_validation_label = rfc.predict(X_val)
predict_validation_prob = rfc.predict_proba(X_val)[:,1]

print('accuracy : ', accuracy_score (Y_train, predict_train_label))
print('validation : ', accuracy_score(Y_val, predict_validation_label))
print('\n')

print('roc_auc_score : ', roc_auc_score (Y_train, predict_train_proba))
print('validation : ', roc_auc_score(Y_val, predict_validation_prob))




accuracy :  0.9958839473948399
validation :  0.7563227619429947


roc_auc_score :  0.9998250806523219
validation :  0.738117120942242
